[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/sensioai/blog/blob/master/048_cv_detection/cv_detection.ipynb)

#Laboratorio 6. Detección de objetos con el modelo Sparse R-CNN

DE LAS DIFERENTES ARQUITECTURAS QUE EXISTEN PARA HACER DETECCIÓN DE OBJETOS, PODEMOS CATEGORIZARLAS EN 2 GRUPOS:
1. DETECTORES DE DOS ETAPAS: TENEMOS UNS PRIMERA ETAPA EN LA QUE UN ALGORITMO (RED NEURONAL U OTRO) NOS PROPONE CAJAS -> "AQUÍ HAY UN OBJETO, AQUÍ OTRO...."\
LUEGO HAY UNA SEGUNDA ETAPA QUE SE ENCARGA DE CLASIFICAR LOS OBJETOS QUE HAY DENTRO DE ÉSTAS CAJAS QUE NOS HA PROPUESTO LA PRIMERA ETAPA.\
-ÉSTE TIPO DE ALGORITMOS SON LOS QUE TIENEN MEJOR PRECISIÓN, MÁS ACIERTAN. SIN EMBARGO SON BASTANTE LENTOS, PESADOS DE CALCULAR.\
-POR TANTO, NO SE UTILIZAN PARA APLICACIONES EN TIEMPO REAL.
2. DTECTORES DE UNA ETAPA\
EN UNA SOLA ETAPA NOS DAN YA LAS CAJAS Y LAS CLASIFICACIONES DENTRO DE ELLAS.\
-SON ALGORITMOS MÁS RÁPIDOS, SE PUEDEN UTILIZAR PARA APLICACIONES EN TIEMPO REAL PERO TIENEN MENOS PRECISIÓN QUE LOS DE 2 ETAPAS.

###Actualizar pip

In [ ]:
!pip install --upgrade pip

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 30.7 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 24.1.2
    Uninstalling pip-24.1.2:
      Successfully uninstalled pip-24.1.2


###Instalar dependencias

In [ ]:
!apt-get install -y gcc g++  # Necesario para compilar algunas bibliotecas
!pip install cython

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
g++ is already the newest version (4:11.2.0-1ubuntu1).
g++ set to manually installed.
gcc is already the newest version (4:11.2.0-1ubuntu1).
gcc set to manually installed.
0 upgraded, 0 newly installed, 0 to remove and 49 not upgraded.


###Clonar el repositorio de MMDetection

In [ ]:
!git clone https://github.com/open-mmlab/mmdetection.git
%cd mmdetection

Cloning into 'mmdetection'...
remote: Enumerating objects: 38023, done.
remote: Total 38023 (delta 0), reused 0 (delta 0), pack-reused 38023 (from 1)
Receiving objects: 100% (38023/38023), 63.25 MiB | 22.84 MiB/s, done.
Resolving deltas: 100% (26223/26223), done.
/content/mmdetection


###Instalar MMDetection

In [ ]:
!pip install -r requirements/build.txt  # Instalar requisitos de construcción
!pip install -v -e .  # Instalar MMDetection

Using pip 24.3.1 from /usr/local/lib/python3.10/dist-packages/pip (python 3.10)
Obtaining file:///content/mmdetection
  Running command python setup.py egg_info
  running egg_info
  creating /tmp/pip-pip-egg-info-lgak7gp2/mmdet.egg-info
  writing /tmp/pip-pip-egg-info-lgak7gp2/mmdet.egg-info/PKG-INFO
  writing dependency_links to /tmp/pip-pip-egg-info-lgak7gp2/mmdet.egg-info/dependency_links.txt
  writing requirements to /tmp/pip-pip-egg-info-lgak7gp2/mmdet.egg-info/requires.txt
  writing top-level names to /tmp/pip-pip-egg-info-lgak7gp2/mmdet.egg-info/top_level.txt
  writing manifest file '/tmp/pip-pip-egg-info-lgak7gp2/mmdet.egg-info/SOURCES.txt'
  reading manifest file '/tmp/pip-pip-egg-info-lgak7gp2/mmdet.egg-info/SOURCES.txt'
  reading manifest template 'MANIFEST.in'
  adding license file 'LICENSE'
  writing manifest file '/tmp/pip-pip-egg-info-lgak7gp2/mmdet.egg-info/SOURCES.txt'
  Preparing metadata (setup.py) ... done
  Obtaining dependency information for terminaltables from h

###Instalar MMCV

In [ ]:
!pip install mmcv-full -f https://download.openmmlab.com/mmcv/dist/cu121/torch2.5.0/index.html

Looking in links: https://download.openmmlab.com/mmcv/dist/cu121/torch2.5.0/index.html
  Using cached mmcv-full-1.7.2.tar.gz (607 kB)
  Preparing metadata (setup.py) ... done
  Using cached addict-2.4.0-py3-none-any.whl.metadata (1.0 kB)
  Using cached yapf-0.40.2-py3-none-any.whl.metadata (45 kB)
Using cached addict-2.4.0-py3-none-any.whl (3.8 kB)
Using cached yapf-0.40.2-py3-none-any.whl (254 kB)
  Created wheel for mmcv-full: filename=mmcv_full-1.7.2-cp310-cp310-linux_x86_64.whl size=32414390 sha256=70ec3c1d5810d35119ad81dc88376bf92b1d25f7b644252278c985429b07d5e6
  Stored in directory: /root/.cache/pip/wheels/4e/10/0a/78fbabe3d28cc25866432d4ecd6c9b124b8f0a6f546974bb7e
Successfully built mmcv-full


In [ ]:
!pip install mmengine

###Verificar la instalación

In [ ]:
import mmdet
print(mmdet.__version__)  # Deberías ver la versión instalada

AssertionError: MMCV==1.7.2 is used but incompatible. Please install mmcv>=2.0.0rc4, <2.2.0.

## Dataset Plantas

Vamos a ver como podemos utilizar el modelo `Faster R-CNN` disponible en `torchvision` para generar detecciones. En primer lugar vamos a utilizar el mismo dataset que hemos usado en los vídeos anteriores, el dataset `VOC` (el modelo `Faster R-CNN` de `torchvision` está entrenado en el dataset `COCO`)

In [ ]:
import torch
#print(torch.__version__)
import torchvision
device = "cuda" if torch.cuda.is_available() else "cpu"
device

2.5.0+cu121


'cuda'

In [ ]:
#Colocar path
train = Plantas('./data', download=True)
len(train)

In [ ]:
print(train[0])

In [ ]:
classes = ["background",
            "laurel",
            "limon",
            "membrillo",
            "olivo",
            "papaya",
            "molle",
            "llanten",
            "naranja",
            "cmaracuya",
            "guayaba",
            "aloevera",
            "cedron",
            "floripondio",
            "higuera",
            "nispero",
            "amorseco",
            "boldo",
            "perejil",
            "charanguillo",
            "ortiga"]

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import matplotlib.patheffects as PathEffects
import random

def get_sample(ix):
  img, label = train[ix]
  # print("+++"*10)
  # print(img)
  # print("..."*10)
  # print(label)
  # print("+++"*10)
  img_np = np.array(img)
  anns = label['annotation']['object']
  if type(anns) is not list:
    anns = [anns]
  labels = np.array([voc_classes.index(ann['name']) for ann in anns])
  bbs = [ann['bndbox'] for ann in anns]
  bbs = np.array([[int(bb['xmin']), int(bb['ymin']),int(bb['xmax'])-int(bb['xmin']),int(bb['ymax'])-int(bb['ymin'])] for bb in bbs])
  anns = (labels, bbs)
  return img_np, anns

# def plot_anns(img, anns, ax=None, bg=-1, classes=voc_classes):
#   # anns is a tuple with (labels, bbs)
#   # bbs is an array of bounding boxes in format [x_min, y_min, width, height]
#   # labels is an array containing the label
#   if not ax:
#     fig, ax = plt.subplots(figsize=(10, 6))
#   ax.imshow(img)
#   labels, bbs = anns
#   for lab, bb in zip(labels, bbs):
#     if bg == -1 or lab != bg:
#       x, y, w, h = bb
#       rect = mpatches.Rectangle((x, y), w, h, fill=False, edgecolor='red', linewidth=2)
#       text = ax.text(x, y - 10, classes[lab], {'color': 'red'})
#       text.set_path_effects([PathEffects.withStroke(linewidth=5, foreground='w')])
#       ax.add_patch(rect)

def plot_anns(img, anns, ax=None, bg=-1, classes=voc_classes):
  # anns is a tuple with (labels, bbs)
  # bbs is an array of bounding boxes in format [x_min, y_min, width, height]
  # labels is an array containing the label
  if not ax:
    fig, ax = plt.subplots(figsize=(10, 6))
  ax.imshow(img)
  labels, bbs = anns
  for lab, bb in zip(labels, bbs):
    if bg == -1 or lab != bg:
      bb = torch.tensor(bb)
      x, y, w, h = bb.detach().numpy()
      rect = mpatches.Rectangle((x, y), w, h, fill=False, edgecolor='red', linewidth=2)
      text = ax.text(x, y - 10, classes[lab], {'color': 'red'})
      text.set_path_effects([PathEffects.withStroke(linewidth=5, foreground='w')])
      ax.add_patch(rect)

In [ ]:
img_np, anns = get_sample(3133)
print(img_np.shape)
print("--"*20)
print(anns)

(333, 500, 3)
----------------------------------------
(array([9]), array([[ 90,  74, 192, 230]]))


In [ ]:
r, c = 3, 4
fig = plt.figure(figsize=(4*c, 4*r))
for _r in range(r):
    for _c in range(c):
        ax = plt.subplot(r, c, _r*c + _c + 1)
        ix = random.randint(0, len(train)-1)
        img_np, anns = get_sample(ix)
        plot_anns(img_np, anns, ax)
        plt.axis("off")
plt.tight_layout()
plt.show()

###Descargar el modelo entrenado

In [ ]:
model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=True)
model

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=FasterRCNN_ResNet50_FPN_Weights.COCO_V1`. You can also use `weights=FasterRCNN_ResNet50_FPN_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/fasterrcnn_resnet50_fpn_coco-258fb6c6.pth" to /root/.cache/torch/hub/checkpoints/fasterrcnn_resnet50_fpn_coco-258fb6c6.pth
100%|██████████| 160M/160M [00:02<00:00, 70.3MB/s]


FasterRCNN(
  (transform): GeneralizedRCNNTransform(
      Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
      Resize(min_size=(800,), max_size=1333, mode='bilinear')
  )
  (backbone): BackboneWithFPN(
    (body): IntermediateLayerGetter(
      (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (bn1): FrozenBatchNorm2d(64, eps=0.0)
      (relu): ReLU(inplace=True)
      (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
      (layer1): Sequential(
        (0): Bottleneck(
          (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn1): FrozenBatchNorm2d(64, eps=0.0)
          (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn2): FrozenBatchNorm2d(64, eps=0.0)
          (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn3): FrozenBatchNorm2d(256, eps=0.0)
          (relu): ReLU(

Siguiendo la documentación, podemos usar este modelo en inferencia pasándole una lista de imágenes normalizadas entre 0-1 con las dimensiones `[C, H, W]` (pueden ser imágenes de diferente tamaño, el modelo ya se encarga del *resize*).

In [ ]:
img_np, anns = get_sample(4445)
print(img_np, anns)

[[[193 215 229]
  [193 215 229]
  [194 215 232]
  ...
  [228 243 246]
  [228 243 246]
  [228 243 246]]

 [[193 215 229]
  [193 215 229]
  [194 215 232]
  ...
  [228 243 246]
  [228 243 246]
  [226 244 246]]

 [[194 216 230]
  [193 215 229]
  [194 215 232]
  ...
  [229 244 247]
  [227 245 247]
  [227 245 247]]

 ...

 [[146 130 117]
  [147 131 118]
  [146 130 117]
  ...
  [123 110 102]
  [125 112 104]
  [119 106  98]]

 [[145 131 118]
  [142 128 115]
  [137 123 110]
  ...
  [127 115 103]
  [131 119 107]
  [129 117 105]]

 [[142 130 116]
  [141 129 115]
  [135 123 109]
  ...
  [128 116 102]
  [126 114 100]
  [129 117 103]]] (array([1]), array([[ 39, 119, 454, 133]]))


In [ ]:
plot_anns(img_np, anns)
plt.show()

MODO EVALUACIÓN:\
-PASAMOS UNA LISTA DE IMÁGENES, ÉSTAS IMÁGENES TIENEN QUE ESTAR NORMALIZADAS, LA PRIMERA DIMENSIÓN TIENE QUE SER LA DE LOS CANALES (COLOR) -> permute()\
-TODO ESO LE PASAMOS AL MODELO PARA QUE NOS DE LOS OUTPUTS\
-VISUALIZACIÓN DE OUTPUTS: TENEMOS UNA LISTA Y CADA ELEMENTO DE ÉSTA LISTA ES UN DICT. EN ÉSTE CASO LE HEMOS DADO SOLO UNA IMAGEN Y SÓLO TENEMOS UN DICT.

In [ ]:
model.eval()
outputs = model([torch.tensor(img_np / 255.).permute(2,0,1).float()])
outputs

[{'boxes': tensor([[ 38.9765, 123.2571, 463.2631, 244.7916],
          [394.6673, 239.3306, 417.8829, 248.7579],
          [ 83.8890, 211.9992,  97.7212, 245.2696],
          [391.7524, 240.2934, 407.5306, 248.3502],
          [297.7431, 236.6284, 325.2031, 250.0018],
          [ 33.2589, 174.6996, 368.3978, 247.6585],
          [297.8233, 236.4347, 324.7085, 249.9443],
          [ 88.0805, 211.7508,  96.9347, 233.4872],
          [209.9664, 119.5987, 489.9711, 232.0793],
          [276.2681, 239.4820, 298.6529, 247.7382],
          [  1.0193, 227.8070,  24.9193, 240.9534],
          [ 76.8103, 229.2830,  82.8956, 246.1568],
          [  9.1523, 234.0433,  16.6872, 241.9108],
          [268.0753, 238.7643, 301.0202, 249.1797],
          [392.7399, 238.6301, 419.9506, 249.2913]], grad_fn=<StackBackward0>),
  'labels': tensor([ 5,  3,  1,  3,  3,  5,  8,  1,  5,  3, 15,  1, 11,  8,  8]),
  'scores': tensor([0.9950, 0.9116, 0.8834, 0.5434, 0.3926, 0.3855, 0.3645, 0.2069, 0.1535,
         

A la salida recibimos una lista con las detecciones de cada imagen (en nuestro caso solo una). Cada elemento de la lista es un `dict` con las cajas, etiquetas y las probabilidades asignadas a cada objeto detectado (que podemos filtrar para quedarnos, por ejemplo, con aquellas detecciones con probabilidad superior a un cierto valor). Es importante recordar que estas etiquetas corresponden al dataset `COCO`.

-EL PRIMER CAMPO ES EL DE BOXES\
-LUEGO TENEMOS LABELS QUE SON LAS ETIQUETAS\
-Y LOS SCORES QUE SERÍA LA PROBABILIDAD ASIGNADA A CADA UNA DE ÉSTAS ETIQUETAS.\
-EN BOXES TENEMOS UNA LISTA CON TODAS LAS CAJAS QUE HA DETECTADO DENTRO DE ESA IMAGEN PERO TENÍAMOS SÓLO UNA -> UN AVIÓN; POR QUÉ NOS HA DADO TANTAS?\
-PORQUE PUEDE DETECTAR MUCHAS COSAS, ALGUNAS BIEN, ALGUNAS MAL. PARA ELLO HAY QUE VENIR A LOS SCORES: MUCHOS DE ELLOS TIENEN VALORES MUY BAJOS, PODEMOS FILTRAR Y DECIR "DAME SÓLO LOS QUE TENGAN UNA PROBABILIDAD MÁS GRANDE AL 90 POR CIENTO POR EJEMPLO".\
-EN ÉSTE CASO, VOY A ELEGIR LA PRIMERA QUE TIENE LA ETIQUETA 5 Y\
OJO-> ÉSTA ETIQUETA CORRESPONDE AL DATASET CON EL QUE FUE ENTRENADA ESTA RED ORIGINALMENTE QUE NO ES EL MISMO.\
-ESTAMOS DANDO IMÁGENES DEL DATASET VOC PERO ÉSTA RED FUE ENTRENADA CON EL DATASET COCO QUE TIENE 90 CLASES -> VER LA LISTA ABAJO: 5 ES EL airplane:


####FUNCIÓN predict()
-BÁSICAMENTE SACA TODAS ÉSTAS CAJAS Y PERMITE VISUALIZAR.\
-PUEDES EJECUTAR ESTA CELDA MUCHAS VECES CON DIFERENTES IMÁGENES Y VER QUE TE DA.\
-DATASET COCO: TIENE MUCHAS MÁS CLASES QUE EL DATASET VOC. SI COMPARAS LAS DETECCIONES QUE TE DA CON LAS ORIGINALES, VERÁS QUE HAY MUCHOS OBJETOS QUE DETECTA BIEN PERO QUE NO ESTABAN EN EL DATASET ORIGINAL.
-EN EL CÓDIGO DE ABAJO, ME ESTOY QUEDANDO CON TODAS LAS DETECCIONES QUE TENGAN UNA PROBABILIDAD MAYOR AL 80%.\
threshold=0.8  (zrésh hold)\
ÉSTE ES UN HIPERPARÁMETRO CON EL QUE TIENES QUE JUGAR PARA VER CUÁL FUNCIONA MEJOR.


In [ ]:
def predict(img, threshold=0.8):

    model.eval()
    outputs = model([torch.tensor(img_np / 255.).permute(2,0,1).float()])
    # nos quedamos con la primera detección
    bb = outputs[0]['boxes'][0].long().tolist()
    bbs = [[bb[0], bb[1], bb[2]-bb[0], bb[3]-bb[1]] for o in outputs for bb, score in zip(o['boxes'], o['scores']) if score > threshold]
    labels = [lab for o in outputs for lab, score in zip(o['labels'], o['scores']) if score > threshold]

    return labels, bbs


In [ ]:
ix = random.randint(0, len(train)-1)
img_np, anns = get_sample(ix)
plot_anns(img_np, anns)
plt.show()

In [ ]:
anns = predict(img_np)
#plot_anns(img_np, anns, classes=COCO_INSTANCE_CATEGORY_NAMES)
plot_anns(img_np, anns, classes=SPARSE_INSTANCE_CATEGORY_NAMES)
plt.show()